# Práctico Ensembles

Diplomado Machine Learning Aplicado, PUC Chile

**Profesor:** Vicente Domínguez

**Alumno:** `Sebastián Ignacio Latorre Diaz`

En este práctico vamos a utilizar la biblioteca de Python [fastFM](https://github.com/ibayer/fastFM) para recomendación utilizando máquinas de factorización.

En este caso utilizaremos un dataset de cervezas, donde además de incluir interacciones de usuarios con los items agregaremos features de cervezas.

instalación de fastFM (puede demorar un poco)

In [2]:
%%capture
!apt-get install python-dev libopenblas-dev
!git clone --recursive https://github.com/ibayer/fastFM.git
%cd fastFM
!pip install -r ./requirements.txt
!make
!pip install .
%cd ..

importamos librerias necesarias

In [3]:
import numpy as np
import pandas as pd
import fastFM
from fastFM.datasets import make_user_item_regression
from sklearn.model_selection import train_test_split
from fastFM import sgd
from fastFM import als
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from scipy.sparse import csc_matrix
from fastFM import mcmc
import functools as fct
import itertools as itools
import random, scipy

# Cargamos los datos

In [4]:
!wget http://jmcauley.ucsd.edu/cse190/data/beer/beer_50000.json

--2024-08-11 20:32:05--  http://jmcauley.ucsd.edu/cse190/data/beer/beer_50000.json
Resolving jmcauley.ucsd.edu (jmcauley.ucsd.edu)... 137.110.160.73
Connecting to jmcauley.ucsd.edu (jmcauley.ucsd.edu)|137.110.160.73|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61156124 (58M) [application/json]
Saving to: ‘beer_50000.json’

beer_50000.json     100%[===================>]  58.32M  73.7MB/s    in 0.8s    

2024-08-11 20:32:06 (73.7 MB/s) - ‘beer_50000.json’ saved [61156124/61156124]



In [5]:
import pandas as pd

appearances = []
tastes = []
names = []
ratings = []
users = []
items = []
aromas = []
styles = []

with open('beer_50000.json') as f:

  for line in f:
    l = line.replace('\n' , '')
    formated_l = eval(l)

    appearance = formated_l['review/appearance']
    taste = formated_l['review/taste']
    name = formated_l['beer/name']
    rating = formated_l['review/overall']
    user_id = formated_l['beer/brewerId']
    item_id = formated_l['beer/beerId']
    aroma = formated_l['review/aroma']
    style = formated_l['beer/style']


    appearances.append(appearance)
    tastes.append(taste)
    names.append(name)
    ratings.append(rating)
    users.append(user_id)
    items.append(item_id)
    aromas.append(aroma)
    styles.append(style)



df = pd.DataFrame()

df['user_id'] = users #
df['item_id'] = items #
df['rating'] = ratings #
df['aroma'] = aromas #
df['taste'] = tastes #
df['appearance'] = appearances #
df['style'] = styles
df['name'] = names

style_to_id = {style: id for id, style in enumerate(df['style'].unique())}
df['styleID'] = df['style'].map(style_to_id)

df


,user_id,item_id,rating,aroma,taste,appearance,style,name,styleID
0,10325,47986,1.5,2.0,1.5,2.5,Hefeweizen,Sausa Weizen,0
1,10325,48213,3.0,2.5,3.0,3.0,English Strong Ale,Red Moon,1
2,10325,48215,3.0,2.5,3.0,3.0,Foreign / Export Stout,Black Horse Black Beer,2
3,10325,47969,3.0,3.0,3.0,3.5,German Pilsener,Sausa Pils,3
4,1075,64883,4.0,4.5,4.5,4.0,American Double / Imperial IPA,Cauldron DIPA,4
...,...,...,...,...,...,...,...,...,...
49995,394,20539,4.0,4.0,4.0,4.0,American Double / Imperial Stout,Stoudt's Fat Dog (Imperial Oatmeal Stout),24
49996,394,20539,4.0,4.0,4.0,3.5,American Double / Imperial Stout,Stoudt's Fat Dog (Imperial Oatmeal Stout),24
49997,394,20539,3.5,3.5,4.5,4.0,American Double / Imperial Stout,Stoudt's Fat Dog (Imperial Oatmeal Stout),24
49998,394,20539,4.0,4.0,4.5,4.0,American Double / Imperial Stout,Stoudt's Fat Dog (Imperial Oatmeal Stout),24


Tenemos features como
- sabor
- aroma
- apariencia

# Funciones auxiliares

In [6]:
def get_single_entries_in_fm_input_format(data, itemlist):

    '''Cree el formato de entrada necesario (datos, (fila, columna)) para la matriz csc para
    las entradas individuales en los datos. Cada entrada ocuparía una fila. Esto significa que
    daría como resultado una matriz csc con dimensión (| datos | x | lista de elementos |).
    '''

    column = len(itemlist)
    row = len(data)
    shape = (row, column)

    row_inds = np.zeros(len(data), dtype=int)
    col_inds = np.zeros(len(data), dtype=int)
    datalist = np.zeros(len(data), dtype=float)

    for i in range(len(data)):
        item = data[i]
        val = 1
        datalist[i] = val

        # ubica su posición en la lista de elementos, arroja un error si el elemento no es un
        # artículo posible
        col_ind = np.where(itemlist==item)[0]

        # no deben ser elementos duplicados en la lista de elementos
        assert len(col_ind) == 1
        col_ind = col_ind[0]
        row_ind = i

        col_inds[i] = col_ind
        row_inds[i] = row_ind

    return datalist, row_inds, col_inds, shape


def get_multi_entries_in_fm_input_format(data, itemlist, norm_func=None):

    '''Cree el formato de entrada necesario (datos, (fila, columna)) para la matriz csc para
    las entradas múltiples en los datos. Cada conjunto de entradas múltiples ocuparía una fila.
    Esto significa que daría como resultado una matriz csc con dimensión
    (| conjuntos de entradas en datos | x | lista de elementos |).
    '''

    column = len(itemlist)

    # número de conjuntos de entradas en los datos
    row = len(data)
    shape = (row, column)

    # numero de datos
    num_of_data = fct.reduce(lambda x, y: x + len(y), data, 0)
    row_inds = np.zeros(num_of_data, dtype=int)
    col_inds = np.zeros(num_of_data, dtype=int)
    datalist = np.zeros(num_of_data, dtype=float)
    cnt = 0
    for i in range(len(data)):
        multi_entry = data[i]

        if norm_func != None:
            # función que recibe el tamaño del multi_entry para decidir cómo normalizarlo
            val = norm_func(len(multi_entry))
        else:
            # asignación de valor binario por defecto
            val = 1 if len(multi_entry) > 0 else 0

        # para cada entrada en multi_entry, ubique su posición en la lista de elementos,
        # arroja error si el elemento no es un elemento posible
        # todas las entradas permanecen en la misma fila
        row_ind = i
        for item in multi_entry:
            col_ind = np.where(itemlist==item)[0]
            assert len(col_ind) == 1
            col_ind = col_ind[0]

            datalist[cnt] = val
            col_inds[cnt] = col_ind
            row_inds[cnt] = row_ind

            # actualiza contador
            cnt += 1

    return datalist, row_inds, col_inds, shape


# Modelo que incluye features de las cervezas:
- Sabor
- Aroma
- StyleID

## conversión de los datos

In [7]:
beerlist = df.sort_values('item_id')['item_id'].unique()
userlist = df.sort_values('user_id')['user_id'].unique()

tastelist = df.sort_values('taste')['taste'].unique()
aromalist = df.sort_values('aroma')['aroma'].unique()

# agregarlo aqui
styleIDlist = df.sort_values('styleID')['styleID'].unique()

# usuarios que dieron ratings
user_data = df['user_id'].values

# items que recibieron ratings
beer_data = df['item_id'].values

# data de sabor de la cerveza
tastes_data = df['taste'].values

# data de aroma de la cerveza
aroma_data = df['aroma'].values

# data de styleID de la cerveza
styleID_data = df['styleID'].values

# target vector: ratings
rating_data = df['rating'].values


# convertir a formato fastFM utilizando funciones de arriba
user_datalist, user_row_inds, user_col_inds, user_shape = get_single_entries_in_fm_input_format(data=user_data,
                                                                                                itemlist=userlist)

beer_datalist, beer_row_inds, beer_col_inds, beer_shape = get_single_entries_in_fm_input_format(data=beer_data,
                                                                                                   itemlist=beerlist)

taste_datalist, taste_row_inds, taste_col_inds, taste_shape = get_single_entries_in_fm_input_format(data=tastes_data,
                                                                                                   itemlist=tastelist)

aroma_datalist, aroma_row_inds, aroma_col_inds, aroma_shape = get_single_entries_in_fm_input_format(data=aroma_data,
                                                                                                   itemlist=aromalist)

styleID_datalist, styleID_row_inds, styleID_col_inds, styleID_shape = get_single_entries_in_fm_input_format(data=styleID_data,
                                                                                                   itemlist=styleIDlist)

# Concatena las dos columnas cambiando los índices de las columnas relacionadas con beer.
# cambiar por el número de columnas en las columnas de usuario
shift_by = len(userlist)
beer_col_inds += shift_by
beer_col_inds += shift_by

# concatena los datos (agregamos item_tastes)
datalist = np.append(user_datalist, [beer_datalist, taste_datalist , aroma_datalist, styleID_datalist ])
row_inds = np.append(user_row_inds, [beer_row_inds, taste_row_inds , aroma_row_inds, styleID_row_inds  ])
col_inds = np.append(user_col_inds, [beer_col_inds,taste_col_inds , aroma_col_inds, styleID_col_inds ])

print('User feature set shape: ', user_shape[0])
print('Item feature set shape: ',beer_shape[0])
print('Taste feature set shape: ',taste_shape[0])
print('Aroma feature set shape: ',aroma_shape[0])
print('Style feature set shape: ',styleID_shape[0])

assert user_shape[0] == beer_shape[0]
shape = (user_shape[0], user_shape[0] + beer_shape[0] + taste_shape[0] + aroma_shape[0] + styleID_shape[0])
print('Dimension of FM input: {}'.format(shape))

X = csc_matrix((datalist, (row_inds, col_inds)), shape=shape)
y = rating_data

User feature set shape:  50000
Item feature set shape:  50000
Taste feature set shape:  50000
Aroma feature set shape:  50000
Style feature set shape:  50000
Dimension of FM input: (50000, 250000)


In [8]:
# split train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 123)

## Entrenamiento:
Metaparámetros
- **n_iter (int, optional)** The number of interations of individual samples.
- **init_std (float, optional):** Sets the stdev for the initialization of the parameter
- **random_state (int, optional):** The seed of the pseudo random number generator that initializes the parameters and mcmc chain.
- **rank (int):**The rank of the factorization used for the second order interactions.
- **l2_reg_w (float):** L2 penalty weight for linear coefficients.
- **l2_reg_V (float):** L2 penalty weight for pairwise coefficients.
- **l2_reg (float):** L2 penalty weight for all coefficients (default=0).
- **step_size (float):** Stepsize for the SGD solver, the solver uses a fixed step size and might require a tunning of the number of iterations n_iter.


In [9]:
X_test

<12500x250000 sparse matrix of type '<class 'numpy.float64'>'
	with 56272 stored elements in Compressed Sparse Row format>

In [10]:
fm_sgd = sgd.FMRegression(n_iter=10000000, init_stdev=0.01, rank=30, random_state=123,
                              l2_reg_w=0.1, l2_reg_V=0.5, step_size=0.01)

fm_sgd.fit(X_train, y_train)

y_pred_sgd = fm_sgd.predict(X_test)

In [11]:
y_pred_sgd

array([4.33687756, 4.06460724, 3.19204762, ..., 4.168398  , 4.11235921,
       3.82671696])

In [12]:
error_sgd = mean_squared_error(y_test, y_pred_sgd)
print('Mean squared error: {}'.format(error_sgd))

Mean squared error: 0.23754552564320627


# Modelo que no incluye features adicionales (solo interacciones de rating)

In [13]:
beerlist = df.sort_values('item_id')['item_id'].unique()
userlist = df.sort_values('user_id')['user_id'].unique()

# usuarios que dieron ratings
user_data = df['user_id'].values

# items que recibieron ratings
beer_data = df['item_id'].values

# target vector: ratings
rating_data = df['rating'].values

# convertir a formato fastFM utilizando funciones de arriba
user_datalist, user_row_inds, user_col_inds, user_shape = get_single_entries_in_fm_input_format(data=user_data,
                                                                                                itemlist=userlist)

beer_datalist, beer_row_inds, beer_col_inds, beer_shape = get_single_entries_in_fm_input_format(data=beer_data,
                                                                                                   itemlist=beerlist)


# Concatena las dos columnas cambiando los índices de las columnas relacionadas con beer.
# cambiar por el número de columnas en las columnas de usuario
shift_by = len(userlist)
beer_col_inds += shift_by
beer_col_inds += shift_by

# concatena los datos
datalist = np.append(user_datalist, [beer_datalist])
row_inds = np.append(user_row_inds, [beer_row_inds])
col_inds = np.append(user_col_inds, [beer_col_inds])

# asegúrese de que ambos conjuntos de características tengan el mismo número de filas
print('User feature set shape: {}\nItem feature set shape: {}'.format(user_shape[0], beer_shape[0]))

assert user_shape[0] == beer_shape[0]
shape = (user_shape[0], user_shape[0] + beer_shape[0])
print('Dimension of FM input: {}'.format(shape))

X = csc_matrix((datalist, (row_inds, col_inds)), shape=shape)
y = rating_data

User feature set shape: 50000
Item feature set shape: 50000
Dimension of FM input: (50000, 100000)


In [14]:
# split train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 123)

## Entrenamiento:
Metaparámetros
- **n_iter (int, optional)** The number of interations of individual samples.
- **init_std (float, optional):** Sets the stdev for the initialization of the parameter
- **random_state (int, optional):** The seed of the pseudo random number generator that initializes the parameters and mcmc chain.
- **rank (int):**The rank of the factorization used for the second order interactions.
- **l2_reg_w (float):** L2 penalty weight for linear coefficients.
- **l2_reg_V (float):** L2 penalty weight for pairwise coefficients.
- **l2_reg (float):** L2 penalty weight for all coefficients (default=0).
- **step_size (float):** Stepsize for the SGD solver, the solver uses a fixed step size and might require a tunning of the number of iterations n_iter.


In [15]:
fm_sgd = sgd.FMRegression(n_iter=10000000, init_stdev=0.01, rank=30, random_state=123,
                              l2_reg_w=0.1, l2_reg_V=0.5, step_size=0.01)

fm_sgd.fit(X_train, y_train)

y_pred_sgd = fm_sgd.predict(X_test)

In [16]:
y_pred_sgd

array([2.72151159, 3.75494015, 3.67934752, ..., 3.69131693, 4.06286913,
       3.34234827])

## predicción:

In [17]:
error_sgd = mean_squared_error(y_test, y_pred_sgd)
print('Mean squared error: {}'.format(error_sgd))

Mean squared error: 0.42825771844376626


# **ACTIVIDAD**
Comente los resultados
- Probar agregar feature de **apariencia** (`appearance`) mejora el desempeño del modelo con respecto al que incorpora solo los otros features y al modelo que incopora solo interacciones de usuario e item en terminos de error cuadratico medio. (3 ptos)
- Pruebe modificando el numero de dimensiones latentes (`rank`) 50,100, 200 para el **modelo que tiene todos los features**. Reportar la cantidad de factores latentes optimos. (3 ptos)

`RESPONDER AQUI`

In [18]:
# # configuracion de FMRegression (bajar si es necesario el n_iter)
# fm_sgd = sgd.FMRegression(n_iter=10000000, init_stdev=0.01, rank=50, random_state=123,
#                           l2_reg_w=0.1, l2_reg_V=0.5, step_size=0.01)

# fm_sgd.fit(X_train, y_train) # MATRICES QUE TIENEN INCORPORADOS LOS OTROS FEATURES.

# y_pred_sgd = fm_sgd.predict(X_test) # MATRIZ DE TEST TAMBIEN VA A INCORPORAR LOS FEATURES ADICIONALES.


##Parte 1

- Probar agregar feature de **apariencia** (`appearance`) mejora el desempeño del modelo con respecto al que incorpora solo los otros features y al modelo que incopora solo interacciones de usuario e item en terminos de error cuadratico medio. (3 ptos)

In [25]:
beerlist = df.sort_values('item_id')['item_id'].unique()
userlist = df.sort_values('user_id')['user_id'].unique()

tastelist = df.sort_values('taste')['taste'].unique()
aromalist = df.sort_values('aroma')['aroma'].unique()
appearancelist = df.sort_values('appearance')['appearance'].unique()

# agregarlo aqui
styleIDlist = df.sort_values('styleID')['styleID'].unique()

# usuarios que dieron ratings
user_data = df['user_id'].values

# items que recibieron ratings
beer_data = df['item_id'].values

# data de sabor de la cerveza
tastes_data = df['taste'].values

# data de aroma de la cerveza
aroma_data = df['aroma'].values

# data de apariencia de la cerveza
appearance_data = df['appearance'].values

# data de styleID de la cerveza
styleID_data = df['styleID'].values

# target vector: ratings
rating_data = df['rating'].values


# convertir a formato fastFM utilizando funciones de arriba
user_datalist, user_row_inds, user_col_inds, user_shape = get_single_entries_in_fm_input_format(data=user_data,
                                                                                                itemlist=userlist)

beer_datalist, beer_row_inds, beer_col_inds, beer_shape = get_single_entries_in_fm_input_format(data=beer_data,
                                                                                                   itemlist=beerlist)

taste_datalist, taste_row_inds, taste_col_inds, taste_shape = get_single_entries_in_fm_input_format(data=tastes_data,
                                                                                                   itemlist=tastelist)

aroma_datalist, aroma_row_inds, aroma_col_inds, aroma_shape = get_single_entries_in_fm_input_format(data=aroma_data,
                                                                                                   itemlist=aromalist)

appearance_datalist, appearance_row_inds, appearance_col_inds, appearance_shape = get_single_entries_in_fm_input_format(data=appearance_data,
                                                                                                   itemlist=appearancelist)

styleID_datalist, styleID_row_inds, styleID_col_inds, styleID_shape = get_single_entries_in_fm_input_format(data=styleID_data,
                                                                                                   itemlist=styleIDlist)

# Concatena las dos columnas cambiando los índices de las columnas relacionadas con beer.
# cambiar por el número de columnas en las columnas de usuario
shift_by = len(userlist)
beer_col_inds += shift_by
beer_col_inds += shift_by

# concatena los datos (agregamos item_tastes)
datalist = np.append(user_datalist, [beer_datalist, taste_datalist , aroma_datalist, appearance_datalist, styleID_datalist ])
row_inds = np.append(user_row_inds, [beer_row_inds, taste_row_inds , aroma_row_inds, appearance_row_inds, styleID_row_inds  ])
col_inds = np.append(user_col_inds, [beer_col_inds,taste_col_inds , aroma_col_inds, appearance_col_inds, styleID_col_inds ])

print('User feature set shape: ', user_shape[0])
print('Item feature set shape: ',beer_shape[0])
print('Taste feature set shape: ',taste_shape[0])
print('Aroma feature set shape: ',aroma_shape[0])
print('Appearance feature set shape: ',appearance_shape[0])
print('Style feature set shape: ',styleID_shape[0])

assert user_shape[0] == beer_shape[0]
shape = (user_shape[0], user_shape[0] + beer_shape[0] + taste_shape[0] + aroma_shape[0] + appearance_shape[0] + styleID_shape[0])
print('Dimension of FM input: {}'.format(shape))

X = csc_matrix((datalist, (row_inds, col_inds)), shape=shape)
y = rating_data

User feature set shape:  50000
Item feature set shape:  50000
Taste feature set shape:  50000
Aroma feature set shape:  50000
Appearance feature set shape:  50000
Style feature set shape:  50000
Dimension of FM input: (50000, 300000)


In [27]:
# split train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 123)

In [28]:
X_test

<12500x300000 sparse matrix of type '<class 'numpy.float64'>'
	with 63769 stored elements in Compressed Sparse Row format>

In [29]:
fm_sgd = sgd.FMRegression(n_iter=10000000, init_stdev=0.01, rank=30, random_state=123,
                              l2_reg_w=0.1, l2_reg_V=0.5, step_size=0.01)

fm_sgd.fit(X_train, y_train)

y_pred_sgd = fm_sgd.predict(X_test)

In [30]:
y_pred_sgd

array([4.39543352, 4.26499833, 3.31323249, ..., 4.27530927, 4.10092503,
       3.9231455 ])

In [31]:
error_sgd = mean_squared_error(y_test, y_pred_sgd)
print('Mean squared error: {}'.format(error_sgd))

Mean squared error: 0.22564361051759224


Respuesta
```
Al probar la adición del feature de apariencia (appearance), hemos visto una
mejora en el desempeño del modelo. A continuación, se comparan los errores
cuadráticos medios (MSE) de los distintos modelos:

Modelo que no incluye features adicionales (solo interacciones de rating)
Mean squared error: 0.42825771844376626

Modelo que incluye features de las cervezas (Sabor, Aroma, StyleID)
Mean squared error: 0.23754552564320627

Modelo que incluye todos los features (Sabor, Aroma, appearance, StyleID)
Mean squared error: 0.22564361051759224

Agregar el feature de apariencia al modelo que ya incluía los otros features
(Sabor, Aroma, StyleID) ha reducido el MSE de 0.2375 a 0.2256. Esto muestra que
la inclusión de la apariencia mejora la precisión del modelo. Por lo tanto, el
modelo que incorpora todos los features (incluyendo apariencia) es el que
ofrece el mejor desempeño en términos de error cuadrático medio.
```

## Parte 2

- Pruebe modificando el numero de dimensiones latentes (`rank`) 50,100, 200 para el **modelo que tiene todos los features**. Reportar la cantidad de factores latentes optimos. (3 ptos)

In [32]:
# Definir valores de rank a probar
ranks = [50, 100, 200]
errors = {}

for rank in ranks:
    fm_sgd = sgd.FMRegression(n_iter=10000000, init_stdev=0.01, rank=rank, random_state=123,
                              l2_reg_w=0.1, l2_reg_V=0.5, step_size=0.01)
    fm_sgd.fit(X_train, y_train) # MATRICES QUE TIENEN INCORPORADOS LOS OTROS FEATURES.
    y_pred_sgd = fm_sgd.predict(X_test) # MATRIZ DE TEST TAMBIEN VA A INCORPORAR LOS FEATURES ADICIONALES.
    error_sgd = mean_squared_error(y_test, y_pred_sgd)
    errors[rank] = error_sgd
    print(f'Rank: {rank}, Mean squared error: {error_sgd}')


Rank: 50, Mean squared error: 0.22564342149570402
Rank: 100, Mean squared error: 0.22564313561414906
Rank: 200, Mean squared error: 0.22564264128014147


Respuesta
```
El modelo con Rank 200 tiene el MSE más bajo, por lo que parece ser el mejor en
cuanto a precisión. Sin embargo, las diferencias en el MSE entre los diferentes
ranks son bastante pequeñas y pueden no ser muy significativas en la práctica.

Si el MSE más bajo con Rank 200 vale la pena el costo adicional y no hay signos
de sobreajuste, entonces el Rank 200 podría ser la mejor opción. Si no,
podríamos optar por un rank más bajo para mantener el modelo más simple y
eficiente.
```

